<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/alpaca_execution_adapter_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install alpaca-trade-api

In [2]:
!pip install python-dotenv

In [3]:
from google.colab import files
uploaded = files.upload()  # Prompts you to upload the .env file


Saving Alpaca_keys.env.txt to Alpaca_keys.env.txt


In [4]:
import os

# Rename to .env so dotenv can recognize it
os.rename("Alpaca_keys.env.txt", ".env")


In [5]:
from alpaca_trade_api.rest import REST
from dotenv import load_dotenv
import os

# Load .env credentials
load_dotenv(override=True)
API_KEY = os.getenv("APCA_API_KEY_ID")
SECRET_KEY = os.getenv("APCA_API_SECRET_KEY")
BASE_URL = "https://paper-api.alpaca.markets"  # or live URL

# Create API object
api = REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

# Check account
account = api.get_account()
print("Account status:", account.status)


Account status: ACTIVE


In [7]:
from datetime import datetime, timedelta
from alpaca_trade_api.rest import REST, TimeFrame

end = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
start = (datetime.utcnow() - timedelta(days=10)).replace(microsecond=0).isoformat() + "Z"

bars = api.get_bars("AAPL", TimeFrame.Day, start=start, end=end, feed="iex")

print(f"Bars returned: {len(bars)}")
for bar in bars:
    print(bar.t.date(), bar.c)


Bars returned: 6
2025-06-23 201.53
2025-06-24 200.3
2025-06-25 201.55
2025-06-26 201.05
2025-06-27 201.05
2025-06-30 205.17


In [14]:
from alpaca_trade_api.rest import REST, TimeFrame
from dotenv import load_dotenv
from datetime import datetime, timedelta
import os
import time

# === Trading Logic with Max Attempts ===
max_attempts = 10
attempts = 0

while attempts < max_attempts:
    try:
        clock = api.get_clock()

        if not clock.is_open:
            print("Market closed. Waiting...")
            time.sleep(60)
            attempts += 1
            continue

        # Check current AAPL position
        try:
            position = api.get_position("AAPL")
            print(f"📈 Already holding AAPL: {position.qty} shares")
            break

        except APIError as e:
            if "position does not exist" in str(e):
                print("Not holding AAPL yet, submitting order...")
                api.submit_order(
                    symbol="AAPL",
                    qty=1,
                    side="buy",
                    type="market",
                    time_in_force="gtc"
                )
                print("Order submitted!")
                break
            else:
                raise

    except KeyboardInterrupt:
        print("Script interrupted by user. Exiting...")
        break

    except Exception as e:
        print(f"Unexpected error: {e}")
        time.sleep(60)
        attempts += 1

if attempts >= max_attempts:
    print("Max attempts reached. Exiting without placing order.")


Market closed. Waiting...
Script interrupted by user. Exiting...


In [12]:
order = api.get_order_by_client_order_id("7a5db325-5939-4332-9637-0e9b2afb556f")
print("Status:", order.status)
print("Filled at:", order.filled_at)
print("Filled qty:", order.filled_qty)


Status: filled
Filled at: 2025-06-30 13:30:08.679004+00:00
Filled qty: 1


In [13]:
positions = api.list_positions()
for position in positions:
    print(position.symbol, position.qty, position.market_value)
history = api.get_portfolio_history(period="1D", timeframe="5Min")
print(history.equity[-5:])  # show last 5 values of equity


AAPL 1 205.17
[100004.64, 100004.28, 100004.51, 100004.25, 100003.09]
